# Simulated Annealing (Имитирано закаляване)

<div class="simulated-annealing-algorithm">
    <img src="images/simulated-annealing-algorithm.png" alt="Simulated Annealing" style="width: 700px; margin: 10px;"/>
</div>

### Характеристики:
- вдъхновен от металургията - залаляване на метал
- итеративен алгоритъм
- евристичен (няма доказана сходимост)
- стохастичен (използване на произволни числа)
- MCMC метод (Markov Chain Monte Carlo)

### Параметри:
- **T, Tmin, Tmax** :: температурата е основен параметър, който определя поведението на алгоритъма
    - в началото, когато T е голямо, алгоритъмът прави разходка в ширина (random walk, )
    - към края, когато T е малко, алгоритъмът експлоатира локален екстремум (hill climbing, обхождане в дълбочина)
- **INIT()** - начална точка / начално решение.
- **NEIGHBOUR(T, best)** - дава следваща точка в зависимост от температурата и настоящата най-добра. Например точка от нормалното разпределение около точка **best** с дисперсия **Т**.
<div class="normal">
    <img src="images/normal.png" alt="Normal Distribution" style="width: 400px;"/>
</div>

- **ACCEPT(T, dE)** - определя с каква вероятност ще приемем по-лоша точка.
$$\displaystyle ACCEPT(T, \Delta E) = \mathrm{e}^{\frac{-\Delta E}{kT}}$$
- **ENERGY(X)** - целевата функция, която се оптимизира. При дискретните проблеми трябва да си измислим такава.
- **COOLING(T)** - правилото, по което се намалява температурата **T**
$$COOLING(T) = \alpha T \hspace{2pc} \alpha = 0.99$$

### Поведение:

- Explore $\displaystyle T \to \infty \hspace{1pc} \lim_{T \to \infty} \frac{-\Delta E}{kT} = 0 \hspace{2pc} \mathrm{e}^{0} = 1$

- Exploit $\displaystyle T \to 0 \hspace{1pc} \lim_{T \to 0} \frac{-\Delta E}{kT} = -\infty \hspace{2pc} \mathrm{e}^{-\infty} \to 0$

### Плюсове:
- лесно адаптиране на алгоритъма към даден проблем (непрекъснат или дискретен)
- не налага никакви ограничения върху целевата функция **ENERGY(X)**
    - няма нужда **ENERGY(X)** да е непрекъсната, може и да е **дискретна**
    - не изисква например да е гладка (да има N непрекъснати производни), производната да е Липшицова или Хесиана да е положително полу-дефинитна матрица, както при градиентните методи
    - няма ограничение за изпъкналост (единствен оптимум)
- паралелизъм / последователно пускане на алгоритъма там, където е приключил


### Минуси:
- евристичен, тоест не е гарантирано, че ще намерим оптимум
- не знаем колко близо сме до оптимума


In [1]:
import operator
import numpy as np
import math
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import matplotlib.animation as animation

%matplotlib nbagg

class SimulatedAnnealing():
    def __init__(self, minimize, T_min, T_max, init,
                 neighbour, energy, accept, cooling, should_log):
        self.compare = operator.lt if minimize else operator.gt
        self.T_min = T_min
        self.neighbour = neighbour
        self.energy = energy
        self.accept = accept
        self.cooling = cooling
        
        self.T = T_max
        self.best = init()
        self.step = 0
        self.should_log = should_log

    def iteration(self):
        next = self.neighbour(self.T, self.best)
        deltaE = self.energy(next) - self.energy(self.best)
        if self.compare(deltaE, 0):
            self.best = next
        elif random.random() < self.accept(self.T, deltaE):
            self.best = next

    def cool(self):
        self.T = self.cooling(self.T, self.best)

    def execute(self):
        while self.T > self.T_min:
            self.iteration()
            self.cool()
            if self.should_log:
                self.log()
            self.step += 1
        if self.should_log:
            print('\nf(' + str(self.best) + ') = ' + str(self.energy(self.best)))
        return self.best
    
    def log(self):
        if(self.step % 100 == 0):
            print(str(self.step) + " T="+str(self.T) +
                  "\tf(" + str(self.best) + ") = " + str(self.energy(self.best)))

# standart configuration parameters and functions for Simulated Annealing
t_min = 1e-5
t_max = 1

def init():
    return np.array([0.1, 1.4])

def neighbour(T, x):
    #return x + random.uniform(-T, T, x.shape)
    return random.normal(x, max(T, 0.1), x.shape)

def accept(T, deltaE, k=0.1):
    return math.exp(-(deltaE)/k/T)

def cooling(T, best):
    return T*0.99

In [2]:
# knapsack problem data set generator

DIM = 60
MAX_VALUE = 100000000
MAX_WEIGHT = 100000000
C = 1000000000
W = np.random.randint(MAX_WEIGHT, size=DIM)
V = np.random.randint(MAX_VALUE, size=DIM)
print(W)
print(V)

[38583282 73241852 35868708 52616819 97448922 74014025 50184742 11259849
  3810507 90750388  9154331  7183977 82920344 63805715 29846554 97394333
 44804970 10435666 40125542 18054937 52662721 46042890 26202023 88974724
 41804948 13790331 12539612 15177684   481569 12112281 25957621 93066524
 16066315 25367896  5038969 27286056 97225355  6191684 79016463  9250484
  2717482  3525208 93637130 29581575 29483436 68650213 73654091  6637760
 36240672 80711627 59667160 61288504  6558061 22843606 48611019 66435972
 51706107 68485231 54616983 50488907]
[19291297 75210491 29167229 61664498 21411487 54949295 81473302 80348438
 54730042 85910253 47277011 19769816 83576086 17429579 23680358 50847737
 21292490  6785409 82846451 90787727 58169151 70983025 33839984 37257927
 74650552 86225220 18599612  1893407 58476213 46378510 88799655 25168139
 24371187 74377204 25477593 16384175 14584623 41367454 60059254 17374172
  4647588 65657959 18905627 92985743 91934739 60345974 84278473 50401380
 20930742 490

In [3]:
from functools import partial


class KnapsackAnnealer:
    
    @staticmethod
    def init(DIM):
        return np.random.randint(2, size=DIM)  # DIM zeros and ones

    
    # naive choice of next knapsack
    @staticmethod
    def neighbour(DIM, cost_fn, T, knapsack):
        new_knapsack = np.copy(knapsack)
        new_knapsack[random.randint(DIM)] ^= 1  # flip the bit
        while cost_fn(new_knapsack) == 0:
            new_knapsack[random.randint(DIM)] ^= 1
        return new_knapsack

    
    @staticmethod
    def cost(W, V, C, knapsack):
        total_weight = sum([W[i] for i, chosen in enumerate(knapsack) if chosen])
        total_value = sum([V[i] for i, chosen in enumerate(knapsack) if chosen])
        return 0 if total_weight > C else total_value

    
    @staticmethod
    def accept(T, deltaE, k=0.1):
        return math.exp(deltaE / k / T)

    
    def __init__(self, weights, values, capacity):
        self.W = weights
        self.V = values
        self.C = capacity
        self.DIM = len(self.W)
        
        init = partial(KnapsackAnnealer.init, self.DIM)
        self.cost = partial(KnapsackAnnealer.cost, self.W, self.V, self.C)
        neighbour = partial(KnapsackAnnealer.neighbour, self.DIM, self.cost)
        
        self.annealer = SimulatedAnnealing(False, t_min, t_max, init, neighbour, self.cost,
                                           KnapsackAnnealer.accept, cooling, False)

        
    def run(self):
        return self.annealer.execute()

In [4]:
# maximization of the 0-1 Knapsack Problem with data taken from
# https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

# DIM = 15
# W = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
# V = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
# C = 750
# OPTIMUM = 1458
# OPTIMAL_KNAPSACK = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

W = [382745, 799601, 909247, 729069, 467902, 44328, 34610, 698150,
     823460, 903959, 853665, 551830, 610856, 670702, 488960, 951111,
     323046, 446298, 931161, 31385, 496951, 264724, 224916, 169684]
V = [825594, 1677009, 1676628, 1523970, 943972, 97426, 69666, 1296457,
     1679693, 1902996, 1844992, 1049289, 1252836, 1319836, 953277,
     2067538, 675367, 853655, 1826027, 65731, 901489, 577243, 466257, 369261]
C = 6404180

annealer = KnapsackAnnealer(W, V, C)
solution = annealer.run()
annealer.cost(solution)

OPTIMUM = 13549094
OPTIMAL_KNAPSACK = [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1]

mismatches = 0
for i, item in enumerate(solution):
    mismatches += 1 if solution[i] != OPTIMAL_KNAPSACK[i] else 0
print(str(mismatches) + " items in the selection mismatch with the optimal solution")
print("{0:.3f}% less optimal then the most optimal solution".format((1 - annealer.cost(solution)/OPTIMUM) * 100))

7 items in the selection mismatch with the optimal solution
0.755% less optimal then the most optimal solution


In [5]:
from itertools import chain

from pydash import py_

# error analysis

def ints(filename):
    return [int(line) for line in open(filename)]

def data(id, type):
    return ints("data/p0{}_{}.txt".format(id, type))

test_data = [{
    "weights": data(i, 'w'),
    "prices": data(i, 'p'),
    "capacity": data(i, 'c')[0],
    "optimum": data(i, 's')
 } for i in range(1, 9)]


def test(data):
    annealer = KnapsackAnnealer(data['weights'], data['prices'], data['capacity'])
    return (1 - annealer.cost(annealer.run()) / annealer.cost(data['optimum']))

deviations = py_.flatten(py_.times(lambda x: py_.chain(test_data).map(test).value(), 10))
total_deviation = sum(deviations) / len(deviations)

print(deviations)
print(total_deviation)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004801097393689946, 0.008689658511484288, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002057613168724326, 0.014188181143329537, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006172839506172867, 0.005561257453819413, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003429355281207136, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00968308286886188, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002057613168724326, 0.009016248614113986, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005486968449931462, 0.015088536547166909, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00137174211248281, 0.010004432768714988, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003429355281207136, 0.008132720903700319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002743484224965731, 0.017075901901632662]
0.0016123761162491214
